In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import pip
pip.main(['install','seaborn'])
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from keras.applications import DenseNet121, VGG19
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, LeakyReLU, GlobalAveragePooling2D, Conv2D, Input
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
modelclass = '50+'
suffix = '50+'
iterations = 5
iterations2 = 5

def CreateModel(learn_rate):
    input_tensor = Input(shape=(256, 256, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for layer in base_model.layers[:10]:
        layer.trainable = False
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learn_rate),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

base_dir = 'Data/'
train_images = np.load(os.path.join(base_dir, 'train_images.npy'))
train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
val_images = np.load(os.path.join(base_dir, 'val_images.npy'))
val_labels = np.load(os.path.join(base_dir, 'val_labels.npy'))

if len(train_images.shape) == 3:
    train_images = np.expand_dims(train_images, axis=-1)
    train_images = np.repeat(train_images, 3, axis=-1)

if len(val_images.shape) == 3:
    val_images = np.expand_dims(val_images, axis=-1)
    val_images = np.repeat(val_images, 3, axis=-1)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels.flatten())
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

print("Initial class weights:", class_weights)
print("Start time: ", datetime.datetime.now())

best_model_path = f'Models/class{suffix}{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')

best_val_loss = float('inf')
for i in range(iterations):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-7)
    model = CreateModel(1e-5)
    print(f"Training iteration {i+1}/{iterations}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbest.keras')

print("Training Complete")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting seaborn

Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)

Requirement already satisfied: numpy!=1.24.0,>=1.20 in /opt/conda/lib/python3.11/site-packages (from seaborn) (1.26.4)

Requirement already satisfied: pandas>=1.2 in /opt/conda/lib/python3.11/site-packages (from seaborn) (2.2.3)

...

154/155 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.3262 - loss: 0.6178

E0000 00:00:1738433148.312135    1460 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433148.520329    1460 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433149.289846    1460 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433149.573296    1460 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.3264 - loss: 0.6173

E0000 00:00:1738433166.338940    1463 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433166.558272    1463 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433168.121840    1463 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738433168.419610    1463 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


155/155 ━━━━━━━━━━━━━━━━━━━━ 79s 357ms/step - accuracy: 0.3265 - loss: 0.6168 - val_accuracy: 0.3874 - val_loss: 0.4823 - learning_rate: 1.0000e-05
Epoch 2/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 203ms/step - accuracy: 0.3960 - loss: 0.4976 - val_accuracy: 0.3793 - val_loss: 0.5023 - learning_rate: 1.0000e-05
Epoch 3/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 33s 213ms/step - accuracy: 0.4012 - loss: 0.4802 - val_accuracy: 0.3753 - val_loss: 0.4768 - learning_rate: 1.0000e-05
Epoch 4/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 204ms/step - accuracy: 0.3905 - loss: 0.4828 - val_accuracy: 0.3671 - val_loss: 0.4828 - learning_rate: 1.0000e-05
Epoch 5/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 33s 214ms/step - accuracy: 0.4389 - loss: 0.4529 - val_accuracy: 0.4097 - val_loss: 0.4522 - learning_rate: 1.0000e-05
Epoch 6/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 33s 214ms/step - accuracy: 0.4469 - loss: 0.4282 - val_accuracy: 0.4280 - val_loss: 0.4404 - learning_rate: 1.0000e-05
Epoch 7/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 205ms/step - 

In [2]:
best_model_path = f'Models/class{suffix}secondary{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
best_val_loss = float('inf')
for i in range (iterations2):
    model = CreateModel(1e-7)
    model.load_weights(f'Models/class_{suffix}{modelclass}_overallbest.keras')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)
    
    print(f"Training iteration {i+1}/{iterations2}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best last val:{val_loss}")
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbestFIN.keras')

/opt/conda/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training iteration 1/5
Epoch 1/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 41s 232ms/step - accuracy: 0.6001 - loss: 0.3347 - val_accuracy: 0.5558 - val_loss: 0.4196 - learning_rate: 1.0000e-07
Epoch 2/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 206ms/step - accuracy: 0.5859 - loss: 0.3395 - val_accuracy: 0.5497 - val_loss: 0.4226 - learning_rate: 1.0000e-07
Epoch 3/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step - accuracy: 0.6066 - loss: 0.3255 - val_accuracy: 0.5538 - val_loss: 0.4231 - learning_rate: 1.0000e-07
Epoch 4/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step - accuracy: 0.5986 - loss: 0.3387 - val_accuracy: 0.5517 - val_loss: 0.4237 - learning_rate: 1.0000e-07
Epoch 5/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step - accuracy: 0.6069 - loss: 0.3264 - val_accuracy: 0.5538 - val_loss: 0.4237 - learning_rate: 1.0000e-08
Epoch 6/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step - accuracy: 0.5909 - loss: 0.3296 - val_accuracy: 0.5538 - val_loss: 0.4236 - learning_rate: 1.0000e-08
Best last val:0.4195625